In [1]:
!pip install sklearn

Looking in indexes: https://pypi.douban.com/simple/


In [2]:
# 导入需要的工具包
import pandas as pd # data processing
import numpy as np
# import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
import warnings
# me
from sklearn.datasets import load_breast_cancer
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('./cardio_train.csv',sep=';')

In [19]:
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [20]:
len(data)

70000

In [21]:
# 数据预处理
data.drop(columns=['id'],inplace=True) #删除id序列
col = ['年龄','性别','身高','体重','高收缩压','低收缩压','胆固醇','葡萄糖','吸烟','饮酒','体力活动','有无心血管疾病']
data.columns = col
data.info() #查看数据基本信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   年龄       70000 non-null  int64  
 1   性别       70000 non-null  int64  
 2   身高       70000 non-null  int64  
 3   体重       70000 non-null  float64
 4   高收缩压     70000 non-null  int64  
 5   低收缩压     70000 non-null  int64  
 6   胆固醇      70000 non-null  int64  
 7   葡萄糖      70000 non-null  int64  
 8   吸烟       70000 non-null  int64  
 9   饮酒       70000 non-null  int64  
 10  体力活动     70000 non-null  int64  
 11  有无心血管疾病  70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.4 MB


In [22]:
# 对于数值型变量的缺失值用中位数进行填充
data[['身高','体重']] = data[['身高','体重']].fillna(data[['身高','体重']].median())
# 对于分类型变量的缺失值用众数进行填充
data['胆固醇'] = data['胆固醇'].fillna(data['胆固醇'].mode())

In [23]:
any(data.isnull())

True

In [24]:
data.head()#说明：年龄的单位是天数

,年龄,性别,身高,体重,高收缩压,低收缩压,胆固醇,葡萄糖,吸烟,饮酒,体力活动,有无心血管疾病
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [25]:
# 计算相关性系数
correlations = data.corr()['有无心血管疾病'].drop('有无心血管疾病')
print(correlations)

年龄      0.238159
性别      0.008109
身高     -0.010821
体重      0.181660
高收缩压    0.054475
低收缩压    0.065719
胆固醇     0.221147
葡萄糖     0.089307
吸烟     -0.015486
饮酒     -0.007330
体力活动   -0.035653
Name: 有无心血管疾病, dtype: float64


In [26]:
# 切分数据集
msk = np.random.rand(len(data))<0.85
# print(np.random.rand(len(data)))
df_train_test = data[msk]
df_val = data[~msk]
X = df_train_test.drop('有无心血管疾病',axis=1)
y = df_train_test['有无心血管疾病']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=70)

In [27]:
# 数据标准化变换
scale = StandardScaler()
scale.fit(X_train)
X_train_scaled = scale.transform(X_train)
scale.fit(X_test)
X_test_scaled = scale.transform(X_test)

In [28]:
gnb=GaussianNB()
gnb.fit(X_train_scaled,y_train)
gnb.score(X_test_scaled,y_test)

0.5854089532144059

In [29]:
user_data = pd.DataFrame({'年龄':[input('年龄/天数')],'性别':[input('性别')],'身高':[input('身高')],'体重':[input('体重/公斤')],'高收缩压':[input('高收缩压')],'低收缩压':[input('低收缩压')],'胆固醇':[input('胆固醇')],'葡萄糖':[input('葡萄糖')],'吸烟':[input('吸烟')],'饮酒':[input('饮酒')],'体力活动':[input('体力活动')]})
if gnb.predict(user_data)[0] == 1:
    print('可能患病')
else:
    print('可能不患病')

年龄/天数0
性别0
身高0
体重/公斤0
高收缩压0
低收缩压0
胆固醇0
葡萄糖0
吸烟0
饮酒0
体力活动0
可能不患病
